In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file (only using concept_region_store_type_data3.csv)
df = pd.read_csv('concept_region_store_type_data3.csv')

# Define regions and store types with order
regions = ['central', 'western', 'eastern', 'ecomm']
store_type_order = ['ecomm', 'mall', 'standalone']  # Order: ecomm first

# ============================================================================
# STEP 1: Transform data into pivot format with region columns
# ============================================================================
df_pivot = df[['CONCEPT', 'STORE_TYPE', 'REGION', 
               'YEAR_TO_DATE_SALES_CURRENT_YEAR', 
               'GROWTH_PERCENTAGE', 
               'ACHIEVEMENT_PERCENTAGE']].copy()

# Sort by concept and store type (with custom order: ecomm, mall, standalone)
df_pivot['store_type_order'] = df_pivot['STORE_TYPE'].map({st: i for i, st in enumerate(store_type_order)})
concept_store_combos = df_pivot[['CONCEPT', 'STORE_TYPE', 'store_type_order']].drop_duplicates().sort_values(['CONCEPT', 'store_type_order'])

result_data_with_totals = []

for _, row in concept_store_combos.iterrows():
    concept = row['CONCEPT']
    store_type = row['STORE_TYPE']
    
    result_row = {'CONCEPT': concept, 'STORE_TYPE': store_type}
    
    # For each region, get the metrics
    for region in regions:
        region_data = df_pivot[(df_pivot['CONCEPT'] == concept) & 
                               (df_pivot['STORE_TYPE'] == store_type) & 
                               (df_pivot['REGION'] == region)]
        
        if not region_data.empty:
            result_row[f'{region}_ytd_sales_cy'] = region_data['YEAR_TO_DATE_SALES_CURRENT_YEAR'].values[0]
            result_row[f'{region}_growth_pct'] = region_data['GROWTH_PERCENTAGE'].values[0]
            result_row[f'{region}_achievement_pct'] = region_data['ACHIEVEMENT_PERCENTAGE'].values[0]
        else:
            result_row[f'{region}_ytd_sales_cy'] = None
            result_row[f'{region}_growth_pct'] = None
            result_row[f'{region}_achievement_pct'] = None
    
    # Add Concept+Store Type totals
    concept_store_data = df[(df['CONCEPT'] == concept) & (df['STORE_TYPE'] == store_type)].iloc[0]
    result_row['combined_ytd_sales'] = concept_store_data['KSA_YTD_SALES_CONCEPT_STORE']
    result_row['combined_growth_pct'] = concept_store_data['KSA_GROWTH_PCT_CONCEPT_STORE']
    result_row['combined_achievement_pct'] = concept_store_data['KSA_ACHIEVEMENT_CONCEPT_STORE']
    
    result_data_with_totals.append(result_row)

result_df_with_totals = pd.DataFrame(result_data_with_totals)

# ============================================================================
# STEP 2: Add total rows (Concept totals after each concept)
# ============================================================================
final_rows = []
concepts_list = result_df_with_totals['CONCEPT'].unique()

for concept in concepts_list:
    # Add all rows for this concept
    concept_rows = result_df_with_totals[result_df_with_totals['CONCEPT'] == concept]
    final_rows.extend(concept_rows.to_dict('records'))
    
    # Add Concept+Region total row
    concept_region_row = {'CONCEPT': concept, 'STORE_TYPE': 'Total'}
    
    for region in regions:
        concept_region_data = df[(df['CONCEPT'] == concept) & (df['REGION'] == region)]
        if not concept_region_data.empty:
            first_row = concept_region_data.iloc[0]
            concept_region_row[f'{region}_ytd_sales_cy'] = first_row['KSA_YTD_SALES_CONCEPT_REGION']
            concept_region_row[f'{region}_growth_pct'] = first_row['KSA_GROWTH_PCT_CONCEPT_REGION']
            concept_region_row[f'{region}_achievement_pct'] = first_row['KSA_ACHIEVEMENT_CONCEPT_REGION']
        else:
            concept_region_row[f'{region}_ytd_sales_cy'] = None
            concept_region_row[f'{region}_growth_pct'] = None
            concept_region_row[f'{region}_achievement_pct'] = None
    
    # Concept totals
    concept_data = df[df['CONCEPT'] == concept].iloc[0]
    concept_region_row['combined_ytd_sales'] = concept_data['KSA_YTD_SALES_CONCEPT']
    concept_region_row['combined_growth_pct'] = concept_data['KSA_GROWTH_PCT_CONCEPT']
    concept_region_row['combined_achievement_pct'] = concept_data['KSA_ACHIEVEMENT_CONCEPT']
    
    final_rows.append(concept_region_row)

# ============================================================================
# STEP 3: Add KSA section (Store Type totals in order: ECOMM, MALL, STANDALONE)
# ============================================================================
for store_type in store_type_order:
    store_region_row = {'CONCEPT': 'KSA', 'STORE_TYPE': store_type.upper()}
    
    for region in regions:
        store_region_data = df[(df['STORE_TYPE'] == store_type) & (df['REGION'] == region)]
        if not store_region_data.empty:
            first_row = store_region_data.iloc[0]
            store_region_row[f'{region}_ytd_sales_cy'] = first_row['KSA_YTD_SALES_REGION_STORE']
            store_region_row[f'{region}_growth_pct'] = first_row['KSA_GROWTH_PCT_REGION_STORE']
            store_region_row[f'{region}_achievement_pct'] = first_row['KSA_ACHIEVEMENT_REGION_STORE']
        else:
            store_region_row[f'{region}_ytd_sales_cy'] = None
            store_region_row[f'{region}_growth_pct'] = None
            store_region_row[f'{region}_achievement_pct'] = None
    
    # Store-level totals
    store_data = df[df['STORE_TYPE'] == store_type]
    if not store_data.empty:
        first_store_row = store_data.iloc[0]
        store_region_row['combined_ytd_sales'] = first_store_row['KSA_YTD_SALES_STORE']
        store_region_row['combined_growth_pct'] = first_store_row['KSA_GROWTH_PCT_STORE']
        store_region_row['combined_achievement_pct'] = first_store_row['KSA_ACHIEVEMENT_STORE']
    else:
        store_region_row['combined_ytd_sales'] = None
        store_region_row['combined_growth_pct'] = None
        store_region_row['combined_achievement_pct'] = None
    
    final_rows.append(store_region_row)

# ============================================================================
# STEP 4: Add Grand Total with calculated Growth % and Achievement %
# ============================================================================
grand_total_row = {'CONCEPT': 'KSA', 'STORE_TYPE': 'Grand Total'}

for region in regions:
    region_data = df[df['REGION'] == region].iloc[0]
    grand_total_row[f'{region}_ytd_sales_cy'] = region_data['KSA_YTD_SALES_REGION']
    grand_total_row[f'{region}_growth_pct'] = region_data['KSA_GROWTH_PCT_REGION']
    grand_total_row[f'{region}_achievement_pct'] = region_data['KSA_ACHIEVEMENT_REGION']

# Calculate grand total for rightmost column
grand_total_ytd_cy = sum([final_rows[-3]['combined_ytd_sales'], 
                          final_rows[-2]['combined_ytd_sales'], 
                          final_rows[-1]['combined_ytd_sales']])

grand_total_ytd_ly = df.groupby('STORE_TYPE')['KSA_LY_YTD_SALES_STORE'].first().sum()
grand_total_budget = df.groupby(['STORE_TYPE', 'REGION'])['BUDGET'].sum().sum()

grand_total_growth_pct = ((grand_total_ytd_cy - grand_total_ytd_ly) * 100) / grand_total_ytd_ly if grand_total_ytd_ly != 0 else None
grand_total_achievement_pct = ((grand_total_ytd_cy - grand_total_budget) * 100 / grand_total_budget) + 100 if grand_total_budget != 0 else None

grand_total_row['combined_ytd_sales'] = grand_total_ytd_cy
grand_total_row['combined_growth_pct'] = grand_total_growth_pct
grand_total_row['combined_achievement_pct'] = grand_total_achievement_pct

final_rows.append(grand_total_row)

# ============================================================================
# STEP 5: Create DataFrame
# ============================================================================
final_df_complete = pd.DataFrame(final_rows)

ordered_columns = ['CONCEPT', 'STORE_TYPE']
for region in regions:
    ordered_columns.extend([f'{region}_ytd_sales_cy', f'{region}_growth_pct', f'{region}_achievement_pct'])
ordered_columns.extend(['combined_ytd_sales', 'combined_growth_pct', 'combined_achievement_pct'])

final_df_complete = final_df_complete[ordered_columns]

print(f"✓ Data formatted successfully")
print(f"Shape: {final_df_complete.shape}")
print(f"\nLast 10 rows:")
print(final_df_complete.tail(10))

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

# Create workbook
wb = Workbook()
ws = wb.active

# Define styles
red_fill = PatternFill(start_color="C00000", end_color="C00000", fill_type="solid")
white_font = Font(color="FFFFFF", bold=True)
center_align = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style='thin', color='000000'),
    right=Side(style='thin', color='000000'),
    top=Side(style='thin', color='000000'),
    bottom=Side(style='thin', color='000000')
)

# Add header row 1: region and store type labels
ws.append(['region', 'store type'] + ['CENTRAL']*3 + ['WESTERN']*3 + ['EASTERN']*3 + ['ECOMM']*3 + ['CONCEPT + STORE TYPE']*3)

# Add header row 2: metric labels
ws.append(['concept', ''] + ['YTD Sales', 'Growth %', 'Achievement %']*4 + ['YTD Sales', 'Growth %', 'Achievement %'])

# Apply red background and white font to header rows
for col in range(1, 18):
    for row in [1, 2]:
        cell = ws.cell(row, col)
        cell.fill = red_fill
        cell.font = white_font
        cell.alignment = center_align
        cell.border = thin_border

# Merge header cells
ws.merge_cells('A1:A2')  # region
ws.merge_cells('B1:B2')  # store type
ws.merge_cells('C1:E1')  # CENTRAL
ws.merge_cells('F1:H1')  # WESTERN
ws.merge_cells('I1:K1')  # EASTERN
ws.merge_cells('L1:N1')  # ECOMM
ws.merge_cells('O1:Q1')  # CONCEPT + STORE TYPE

# Replace None values with 0 in DataFrame
final_df_complete = final_df_complete.fillna(0)

# Add data rows
current_row = 3
current_concept = None
concept_start_row = None
first_row_of_concept = True

for idx, row_data in final_df_complete.iterrows():
    concept = row_data['CONCEPT']
    
    # Track concept changes for merging
    if concept != current_concept:
        # Merge previous concept cells
        if current_concept is not None and concept_start_row is not None:
            ws.merge_cells(f'A{concept_start_row}:A{current_row-1}')
        
        current_concept = concept
        concept_start_row = current_row
        first_row_of_concept = True
    else:
        first_row_of_concept = False
    
    # Add row
    ws.append(row_data.tolist())
    
    # Apply red fill to first column (A) for the first row of each concept
    if first_row_of_concept:
        ws.cell(current_row, 1).fill = red_fill
        ws.cell(current_row, 1).font = white_font
    
    # Apply borders and alignment to all cells in this row
    for col in range(1, 18):
        cell = ws.cell(current_row, col)
        cell.border = thin_border
        cell.alignment = center_align
    
    current_row += 1

# Merge last concept cells
if concept_start_row is not None:
    ws.merge_cells(f'A{concept_start_row}:A{current_row-1}')

# Save workbook
output_file = 'sales_report_final_formatted.xlsx'
wb.save(output_file)

print(f"✓ Excel file created: {output_file}")
print("\nFormatting applied:")
print("  • Header rows with red background and white text")
print("  • First column (concept names) with red background and white text")
print("  • Black borders on all cells")
print("  • Merged cells for concept names")
print("  • Merged cells for region headers")
print("  • Center aligned all cells")
print("  • Blank values filled with 0")